# Benchmark

https://github.com/miloyip/nativejson-benchmark

In [1]:
octave_version = version ()
octave_hg_id   = version ('-hgid')

octave_version = 7.0.0
octave_hg_id = 173807014259


## The JSON libraries under test

- [Octave](https://abdallahshamy.wordpress.com/)
- [octave-rapidjson](https://github.com/Andy1978/octave-rapidjson)
- [JSONio](https://github.com/gllmflndn/JSONio)
- [jsonlab](https://github.com/fangq/jsonlab)

In [2]:
mkdir ('benchmark');
cd ('benchmark');

# The test files

| foo | bar |
| --- | --- |
| baz | bim |

| `citm_catalog.json` | (1,727,204 Byte) | [(source)](https://github.com/RichardHightower/json-parsers-benchmark/blob/master/data/citm_catalog.json) | Structured data with mixed text and numeric. |
| --- | --- | --- | --- |
| `canada.json` | (2,251,060 Byte) | [(source)](https://github.com/mloskot/json_benchmark/blob/master/data/canada.json) | Numeric data set in [GeoJSON](https://en.wikipedia.org/wiki/GeoJSONhttps://en.wikipedia.org/wiki/GeoJSON) format.
| --- | --- | --- | --- |
- `large-file.json` (26,141,343 Byte)
  [(source)](https://github.com/json-iterator/test-data/blob/master/large-file.json)
  Structured data with mixed text and numeric.

In [3]:
if (exist ('citm_catalog.json', 'file') != 2)
  urlwrite ( ...
    'https://github.com/RichardHightower/json-parsers-benchmark/raw/master/data/citm_catalog.json', ...
    'citm_catalog.json');
end

if (exist ('canada.json', 'file') != 2)
  urlwrite ( ...
    'https://github.com/mloskot/json_benchmark/raw/master/data/canada.json', ...
    'canada.json');
end

if (exist ('large-file.json', 'file') != 2)
  urlwrite ( ...
    'https://github.com/json-iterator/test-data/raw/master/large-file.json', ...
    'large-file.json');
end

## The benchmark

In [4]:
function t = benchmark (json_read_fcn, json_write_fcn)
  test_files = {'citm_catalog.json', 'canada.json', 'large-file.json'};
  N = length (test_files);
  t = nan (N, 2);
  for i = 1:N
    json_str = fileread (test_files{i});
    tic ();
    octave_obj = json_read_fcn (json_str);
    t(i,1) = toc ();
    tic ();
    json_str2 = json_write_fcn (octave_obj);
    t(i,2) = toc ();
  end
end

### Matlab (R2020b, prerelease)

The results have been measured on the same machine, wihout JupyterLab.

In [5]:
t.matlab = [
  0.0768, 0.0853;
  0.1510, 0.5405;
  1.2222, 0.6521];

### Octave (7.0.0, development version)

In [6]:
t.octave = benchmark (@jsondecode, @jsonencode);

### octave-rapidjson

**Installation**

In [7]:
if (exist ('octave-rapidjson', 'dir') == 0)
  urlwrite ( ...
    'https://github.com/Andy1978/octave-rapidjson/archive/2d88511712032b14dea4c2272d82249e7547772a.zip', ...
    'octave-rapidjson.zip');
  unzip  ('octave-rapidjson.zip');
  rename ('octave-rapidjson-2d88511712032b14dea4c2272d82249e7547772a', ...
          'octave-rapidjson');
  cd ('octave-rapidjson')
  urlwrite ( ...
    'https://github.com/Tencent/rapidjson/archive/35e480fc4ddf4ec4f7ad34d96353eef0aabf002d.zip', ...
    'rapidjson.zip');
  unzip  ('rapidjson.zip');
  rename ('rapidjson-35e480fc4ddf4ec4f7ad34d96353eef0aabf002d', 'rapidjson');
  mkoctfile -Wall -Wextra -I./rapidjson/include load_json.cc
  mkoctfile -Wall -Wextra -I./rapidjson/include save_json.cc
  cd ('..')
end

**Benchmark run**

In [8]:
addpath ('octave-rapidjson')
t.rapid_json = benchmark (@load_json, @save_json);
rmpath ('octave-rapidjson')

### JSONio

In [9]:
if (exist ('JSONio', 'dir') == 0)
  urlwrite ( ...
    'https://github.com/gllmflndn/JSONio/archive/6c699a315ac2c578864d8b740a061bff47b718bf.zip', ...
    'JSONio.zip');
  unzip  ('JSONio.zip');
  rename ('JSONio-6c699a315ac2c578864d8b740a061bff47b718bf', 'JSONio');
  cd ('JSONio')
  mkoctfile --mex jsonread.c jsmn.c -DJSMN_PARENT_LINKS
  cd ('..')
end

In [10]:
addpath ('JSONio')
%t.jsonio = benchmark (@jsonread, @jsonwrite);
t.jsonio = [ ...
  0.9583,  30.5410;
  6.1333,  17.4022;
  4.3382, 552.8929];
rmpath ('JSONio')

### jsonlab

In [11]:
if (exist ('jsonlab', 'dir') == 0)
  urlwrite ( ...
    'https://github.com/fangq/jsonlab/archive/d0fb684bd43165d312063345bdb795b628b2c679.zip', ...
    'jsonlab.zip');
  unzip  ('jsonlab.zip');
  rename ('jsonlab-d0fb684bd43165d312063345bdb795b628b2c679', 'jsonlab');
end

In [ ]:
addpath ('jsonlab')
%t.jsonlab = benchmark (@loadjson, @savejson);
t.jsonlab = [ ...
   35.6242,  26.0625;
    6.1303,   0.7365;
  372.2456, 601.5318];
rmpath ('jsonlab')

## The results

In [ ]:
graphics_toolkit ('qt')
titles = {'citm\_catalog.json (mixed)', 'canada.json (numeric)', ...
          'large-file.json (mixed)'};

In [ ]:
for i = 1:3
  subplot (3, 1, i);
  bar ([t.matlab(i,:); t.octave(i,:); t.rapid_json(i,:)]');
  legend ({'Matlab (R2020b, pre)', 'Octave (7.0.0, dev)', ...
           'octave-rapidjson'}, 'Location', 'bestoutside');
  xticklabels({'read','write'});
  ylabel ('time in seconds');
  title (titles{i});
end

In [ ]:
for i = 1:3
  subplot (3, 1, i);
  bar ([t.jsonio(i,:); t.jsonlab(i,:)]');
  legend ({'JSONio', 'jsonlab'}, 'Location', 'bestoutside');
  xticklabels({'read','write'});
  ylabel ('time in seconds');
  title (titles{i});
end